In [1]:
import os
import cv2
import time
import json
import pickle
import numpy as np
import torch
from torch.autograd import Variable
from matplotlib import pyplot as plt
from cnn import SegmentationModel as net

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print('Torch:' + torch.__version__)
print('Opencv:' + cv2.__version__)

cuda
Torch:1.1.0
Opencv:4.1.1


In [3]:
from queue import Queue
from threading import Thread

In [4]:
class FPS:
    def __init__(self):
        # store the start time, end time, and total number of frames
        # that were examined between the start and end intervals
        self._start = None
        self._end = None
        self._stoped = False
        self._numFrames = 0

    def start(self):
        # start the timer
        if self._start is None:
            self._start = time.time()
        
    def stop(self):
        # stop the timer
        if not self._stoped:
            self._end = time.time()
            self._stoped = True

    def update(self):
        # increment the total number of frames examined during the
        # start and end intervals
        if not self._stoped and self._start is not None:
            self._numFrames += 1

    def elapsed(self):
        # return the total number of seconds between the start and
        # end interval
        if self._start is not None:
            if self._stoped:
                return (self._end - self._start)
            else:
                return (time.time() - self._start)
            
    def fps(self):
        if self._start is not None:
            # compute the (approximate) frames per second
            return (self._numFrames / self.elapsed())

In [5]:
class Measure(Thread):

    def __init__(self):
        
        Thread.__init__(self, name='Measure')
        self.q = Queue()
        self.running = True
        self.fps = FPS()
        
    def run(self):
        while self.running:
            if not self.q.empty():
                self.fps.start()
                frame_number, plant_dict = self.q.get()
                self.fps.update()
            else:
                time.sleep(0.01)
                    
    def shutdown(self):
        self.running = False
        self.fps.stop()


In [6]:
class Tracking(Thread):
        
    def __init__(self, out_class, w, h, scale):
        self.w = w
        self.h = h
        self.scale = scale
        self.history = {}
        self.n_plantas = 0
        self.speed = 0
        self.last_speed = 0
        self.speed_avg = np.zeros(5)
        
        self.plant_boxes = np.empty([0, 4])
        self.stem_boxes = np.empty([0, 4])
        
        self.sq = out_class.q
        self.q = Queue()
        Thread.__init__(self, name = 'Tracking')
        self.running = True
        self.fps = FPS()
        
    def __bb_iou__(self, boxes1, boxes2):
        x11, y11, x12, y12 = np.split(boxes1, 4, axis=1)
        x21, y21, x22, y22 = np.split(boxes2, 4, axis=1)
        xA = np.maximum(x11, np.transpose(x21))
        yA = np.maximum(y11, np.transpose(y21))
        xB = np.minimum(x12, np.transpose(x22))
        yB = np.minimum(y12, np.transpose(y22))
        interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
        boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
        boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
        iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
        return iou

    def __nms__(self, boxes, overlapThresh = 0.1):
        # if there are no boxes, return an empty list
        if len(boxes) == 0:
            return(np.empty([0, 4]))

        boxes = boxes.astype("float")

        # initialize the list of picked indexes	
        pick = []

        # grab the coordinates of the bounding boxes
        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]

        # compute the area of the bounding boxes and sort the bounding
        # boxes by the bottom-right y-coordinate of the bounding box
        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        # keep looping while some indexes still remain in the indexes
        # list
        while len(idxs) > 0:
            # grab the last index in the indexes list and add the
            # index value to the list of picked indexes
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            # compute the width and height of the bounding box
            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            # compute the ratio of overlap
            overlap = (w * h) / area[idxs[:last]]

            # delete all indexes from the index list that have
            idxs = np.delete(idxs, np.concatenate(([last],
                np.where(overlap > overlapThresh)[0])))

        # return only the bounding boxes that were picked using the
        # integer data type
        return boxes[pick].astype("int64")

    def __get_bboxes__(self, msk, minArea, maxArea, minDst, maxDst):
        contours, hierarchy = cv2.findContours(msk, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        bboxes = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            bbox = cv2.boundingRect(cnt)
            x,y,w,h = np.array(bbox)
            if area >  minArea and area < maxArea:
                if (x > minDst) and ((x+w) < maxDst):
                    bboxes.append([x,y,x+w,y+h])
        bboxes = self.__nms__(np.array(bboxes))
        return(bboxes)

    def __match__(self, old, new):
        areas = None
        if new.size > 0 and old.size > 0 :
            areas = self.__bb_iou__(new, old)
        return(areas)
        
    def __update_speed__(self, dst):
        self.last_speed += dst
        self.last_speed = np.max([-1, np.min([10, self.last_speed])])
        self.speed_avg = np.insert(self.speed_avg, 0, self.last_speed)[:5]
        self.speed = np.mean(self.speed_avg)
        
    def __update__(self, old, new, areas, calc_speed = False):
        tmp = np.empty([0, 4])
        mcrit = np.ones(len(new), dtype=bool)

        if new.size > 0:
            if old.size > 0 :
                ncrit = np.where(areas.max(0) > 0)
                acrit = areas.argmax(0)[ncrit]        
                
                if calc_speed:
                    # Utiliza os bbox correspondentes para calcular a velocidade (px/frame):
                    # Obs: O calculo eh mais estavel com base nos tubetes
                    dst = np.nan
                    try:
                        dst = np.mean(new[acrit, [0,2]] - old[ncrit, [0,2]])
                    except:
                        pass
#                         print('Dst Error')
                        
                    if not np.isnan(dst):
                        self.__update_speed__(dst)

                old[ncrit] = new[acrit]
                mcrit[acrit] = False
            else:
                old = new
            tmp = new[mcrit]
   
        return([old, tmp])

    def get_frame_dict(self, pb, sb):

        frame_str = {}

        for idx, (pbb, sbb) in enumerate(zip(pb, sb)):
            pbb = pbb.copy()
            sbb = sbb.copy()
            pidx = idx + self.n_plantas + 1
            pbb[3] = sbb[1] + 10
            plant_bb = np.int0(pbb * self.scale).tolist()

            xm = int(np.mean(sbb[[0,2]]))
            sbb[[0,2]] = xm + np.array([-10,10])
            sbb[[1,3]] = [sbb[1] - 5, sbb[1] + 5]                    
            stem_bb = np.int0(sbb * self.scale).tolist()
            
            frame_str['P{}'.format(pidx)] = {'Plant':plant_bb, 'Stem':stem_bb}
        return(frame_str)


    def __prep_next__(self, old, tmp):
        old = np.concatenate([old, tmp])
        old = self.__nms__(old)
        if old.size > 0:
            old += np.int0(self.speed * np.ones((len(old), 1)) * np.array([[1,0,1,0]]))
            old = old[old[:,0].argsort()[::-1]]
        return(old)

    def __remove__(self):
        if self.stem_boxes.size > 0:
            # Remove os bbox que estarao fora da imagem, e conta como uma nova planta:
            fcrit = self.stem_boxes[:,2] > self.w
            self.plant_boxes = self.plant_boxes[np.logical_not(fcrit)]
            self.stem_boxes = self.stem_boxes[np.logical_not(fcrit)]
            self.n_plantas += fcrit.sum()         

    def __validate__(self, pb, sb):
        pbb = np.empty([0, 4])
        sbb = np.empty([0, 4])
        if pb.size > 0 and sb.size > 0:
            pb = pb[pb[:,0].argsort()[::-1]]
            sb = sb[sb[:,0].argsort()[::-1]]
            areas = self.__bb_iou__(pb, sb)
            nidx = [(pbix, sbix) for pbix, sbix in enumerate(areas.argmax(1)) if areas.max(1)[pbix] > 0]
            if len(nidx) > 0:
                pbb = np.array([pb[pbix] for pbix, sbix in nidx])
                sbb = np.array([sb[sbix] for pbix, sbix in nidx])
        return(pbb, sbb)
    
            
    def __get_plant_dict__(self, frame, frame_str):
        plant_dict = {}
        for p in frame_str.keys():
            f_bbox = frame_str.get(p)
            pbox = np.int0(f_bbox.get('Plant'))
            plant = frame[pbox[1]:pbox[3], pbox[0]:pbox[2]]
            plant_dict[p] = plant
        return(plant_dict)

        
    def run(self):
        while self.running:
            if not self.q.empty() and self.sq.qsize() < 10:
                self.fps.start()
                frame_number, frame, cat = self.q.get()

                plant_msk = np.isin(cat, [1,2]).astype('uint8')
                new_pb = self.__get_bboxes__(plant_msk, 50, 50000, 0.01*self.w, 0.99*self.w)

                stem_msk = (cat == 2).astype('uint8')
                new_sb = self.__get_bboxes__(stem_msk, 10, 10000, 0.01*self.w, 0.99*self.w)

                new_pb, new_sb = self.__validate__(new_pb, new_sb)

                area_crit = self.__match__(self.stem_boxes, new_sb)
                old_pb, tmp_pb = self.__update__(self.plant_boxes, new_pb, area_crit, calc_speed = False)
                old_sb, tmp_sb = self.__update__(self.stem_boxes, new_sb, area_crit, calc_speed = True)

                frame_str = self.get_frame_dict(old_pb, old_sb)
                self.history['Frame{:04d}'.format(frame_number)] = frame_str
                
                plant_dict = self.__get_plant_dict__(frame, frame_str)
                self.sq.put([frame_number, plant_dict])
                self.fps.update()
                
                new_pb = self.__prep_next__(old_pb, tmp_pb)
                new_sb = self.__prep_next__(old_sb, tmp_sb)
                self.plant_boxes, self.stem_boxes = trk.__validate__(new_pb, new_sb)
                self.__remove__()              

            else:
                time.sleep(0.01)   
        
    def shutdown(self):
        self.running = False
        self.fps.stop()

In [7]:
class TrackingModel(Thread):
        
    def __init__(self, out_class, device):
        self.w = out_class.w
        self.h = out_class.h
        self.device = device
        self.model = self.__init_model__()
        self.sq = out_class.q
        self.q = Queue()
        Thread.__init__(self, name = 'Tracking')
        self.running = True
        self.fps = FPS()
        
    def __init_model__(self):
        n_classes = 4
        model = net.EESPNet_Seg(n_classes, s=0.5, pretrained='', gpus=1)
        model = model.to(self.device)
        model.load_state_dict(torch.load('../data/Trackingff.pth', map_location=self.device))
        model.eval()
        return(model)
        
        
    def __transform__(self, x):
        x = x.astype('float')
        x -= 128
        x /= 35
        x = np.moveaxis(x, 2, 0)
        x = torch.unsqueeze(torch.from_numpy(x), 0)
        x = Variable(x).to(self.device, dtype=torch.float)
        return(x)

    def __colorir__(self, cat):
        h, w = cat.shape[:2]
        msk = np.zeros((h,w,3), dtype = 'uint8')
        msk[cat == 0] = [255,255,255]
        msk[cat == 1] = [0,255,0]
        msk[cat == 2] = [0,0,255]
        msk[cat == 3] = [255,255,255]
        return(msk)


    def __get_cat__(self, frame):
        img = self.__transform__(frame)
        pred, feat = self.model(img)
        pred = pred.cpu().data.numpy()
        pred = np.moveaxis(pred, 1, 3)
        pred = np.squeeze(pred)
        cat = np.argmax(pred, 2).astype('uint8')
        return(cat)

    def run(self):
        while self.running:
            if not self.q.empty() and self.sq.qsize() < 10:
                self.fps.start()
                frame_number, frame, frame_lr = self.q.get()
                cat = self.__get_cat__(frame_lr)
                self.sq.put([frame_number, frame, cat])
                self.fps.update()
            else:
                time.sleep(0.01)
        
    def shutdown(self):
        self.running = False
        self.fps.stop()

In [8]:
class VideoCapture(Thread):

    def __init__(self, out_class, video_file, silent=True):
        
        Thread.__init__(self, name='VideoCapture')
        self.sq = out_class.q       
        self.w = out_class.w
        self.h = out_class.h
        self.frame_number = 0
        self.cap = cv2.VideoCapture(video_file)
        self.running = True
        self.silent = silent
        self.fps = FPS()
        
    def run(self):
        while self.cap.isOpened() and self.running:
            #Se a fila já tiver mais de 10 frames esperamos 0.01s.
            if self.sq.qsize() < 10:
                self.fps.start()
                ret, frame = self.cap.read()
                if ret:
                    frame_lr = cv2.resize(frame, (self.w, self.h))
                    self.sq.put([self.frame_number, frame, frame_lr])   
                    self.frame_number += 1
                    self.fps.update()

            else:
                time.sleep(0.01)
                    
    def shutdown(self):
        self.running = False
        self.cap.release()
        self.fps.stop()


In [9]:
output_height = 192
output_width = 256


input_height = 1080
input_width = 1920


ioh, iow = input_height/output_height, input_width/output_width
scale = np.array([iow, ioh, iow, ioh])

In [10]:
# crtl = Control()
# crtl.start()

msr = Measure()
msr.start()


In [11]:
trk = Tracking(msr, output_width, output_height, scale)
trk.start()


In [12]:
trkm = TrackingModel(trk, device)
trkm.start()


In [13]:
video_file = '/home/rodrigo7/Notebook/Datasets/Eucalyptus2/Videos/20190726.avi'
cap = VideoCapture(trkm, video_file)
cap.start()


In [14]:
msr.fps.fps(), trk.fps.fps(), trkm.fps.fps(), cap.fps.fps()

(None, None, None, 0.0)

In [15]:
trkm.q.qsize(), trk.q.qsize(), msr.q.qsize()

(0, 0, 0)

In [ ]:
cap.shutdown()
trkm.shutdown()
trk.shutdown()
msr.shutdown()